Arm işlemcisinde bulunan systick timerın kullanılması:

In [ ]:
#include "stm32f4xx.h"                  // Device header

int main(){
    
    RCC->AHB1ENR |= RCC_AHB1ENR_GPIODEN;
    GPIOD->MODER |= GPIO_MODER_MODE15_0;
    
    //systick timer devre dışı bırakıldı.
    SysTick->CTRL &= ~SysTick_CTRL_ENABLE_Msk;
    
    //systick interrupt devre dışı bırakıldı.
    SysTick->CTRL &= ~SysTick_CTRL_TICKINT_Msk;
    
    //1 yapılarak sistem clock seçildi.
    //0 yapılsa idi timerın değeri 8 tikte değişecekti.
    SysTick->CTRL |= SysTick_CTRL_CLKSOURCE_Msk;
    
    //Timer'ın saymaya başlayacağı değeri yazıldı.
    SysTick->LOAD |= (16000000UL & SysTick_LOAD_RELOAD_Msk);
    
    //Anlık değeri 0 yapıldı.
    SysTick->VAL |=(0UL & SysTick_LOAD_RELOAD_Msk);
    
    //systick etkinleştirildi. Böylece sayma işlemi başlatıldı.
    SysTick->CTRL |= SysTick_CTRL_ENABLE_Msk;
    
    while(1){
        GPIOD->ODR |= GPIO_ODR_OD15; //LEDi yak
         
        //timerın 16x10^6'dan 0'a kadar saymasini bekle (1 saniye)
        while(!(SysTick->CTRL  & SysTick_CTRL_COUNTFLAG_Msk));
        
        GPIOD->ODR &= ~GPIO_ODR_OD15; //LEDi söndür
        //timerın 16x10^6'dan 0'a kadar saymasini bekle (1 saniye)
        while(!(SysTick->CTRL  & SysTick_CTRL_COUNTFLAG_Msk));
    }

    return 0;
}


Systick timer'ın başlangıç ayarları systick_init fonksiyonu ile yapılıyor. systick_init fonksiyonu parametre olarak timerın saymaya başlayacağı sayıyı alıyor. 

wait_1sn fonksiyonu systick_init fonksiyonunu 16 milyon sayısı ile çağırıyor. Böylece 1 sn bekleme yapılmasını sağlıyor.

wait_n_sn wait 1sn fonksiyonunu çağırıyor. wait_n_sn fonksiyonuna paramtre olarak girilen sayı kadar saniye bekleniliyor.


In [ ]:
#include "stm32f4xx.h"                  // Device header

void systick_init(unsigned int ticks);
void wait_1sn(void);
void wait_n_sn(int n);

int main(){
    
    RCC->AHB1ENR |= RCC_AHB1ENR_GPIODEN;
    GPIOD->MODER |= GPIO_MODER_MODE15_0;
    
    while(1){
        GPIOD->ODR |= GPIO_ODR_OD15;
         wait_n_sn(10);
        
        GPIOD->ODR &= ~GPIO_ODR_OD15;
        wait_n_sn(5);
    }

    return 0;
}


//systick timerin baslangic ayarlari
//ticks degerine 16 milyon yazilirsa timer 1 snde sayar
void systick_init(unsigned int ticks){
    
    //systick timer devre dışı bırakıldı.
    SysTick->CTRL &= ~SysTick_CTRL_ENABLE_Msk;
    
    //systick interrupt devre dışı bırakıldı.
    SysTick->CTRL &= ~SysTick_CTRL_TICKINT_Msk;
    
    //1 yapılarak sistem clock seçildi.
    //0 yapılsa idi timerın değeri 8 tikte değişecekti.
    SysTick->CTRL |= SysTick_CTRL_CLKSOURCE_Msk;
    
    //Timer'ın saymaya başlayacağı değeri yazıldı.
    SysTick->LOAD |= (ticks & SysTick_LOAD_RELOAD_Msk);
    
    //Anlık değeri 0 yapıldı.
    SysTick->VAL |=(0UL & SysTick_LOAD_RELOAD_Msk);
    
    //systick etkinleştirildi. Böylece sayma işlemi başlatıldı.
    SysTick->CTRL |= SysTick_CTRL_ENABLE_Msk;
}

void wait_1sn(void){
    systick_init(16000000);
    while(!(SysTick->CTRL  & SysTick_CTRL_COUNTFLAG_Msk));   
}

void wait_n_sn(int n){
    int i;
    for(i=0;i<n;i++){
        wait_1sn();
    }
}





